In [1]:
import h5py
import os
import re
import numpy as np
from math import sqrt, log10
from matplotlib import pyplot as plt

# load data

In [ ]:
def read_matlab_v73_file(label, batch, frame):
    file_path = f"E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results/Label_{label}/Batch_{batch}/Frame_{frame}/MTD_result.mat"
    with h5py.File(file_path, 'r') as f:
        var_names = list(f.keys())
        data = {}

        for var_name in var_names:
            # HDF5 中的数据是按列优先存储的，而 NumPy 是按行优先，因此需要转置
            if isinstance(f[var_name], h5py.Dataset):
                data[var_name] = np.array(f[var_name]).T

        return data

# visualize

In [ ]:
def check_path(path):
    path = os.path.dirname(path)
    if not os.path.exists(path):
        os.makedirs(path)


def split_scientific_str(num, precision=10):
    """使用字符串格式化将一个数分成科学计数法的尾数和指数两部分"""
    scientific_str = f"{num:.{precision}e}"
    match = re.match(r'([+-]?\d+\.\d+)e([+-]?\d+)', scientific_str)
    if match:
        mantissa = float(match.group(1))
        exponent = int(match.group(2))
        return mantissa, exponent
    return num, 0


def visualize_rd_matrix(data, save_path=None, eps=1e-6, verbose=False):
    """可视化 RD 矩阵"""
    rd_matrix = data['rd_matrix']
    value = np.zeros_like(rd_matrix, dtype=np.float64)
    for i in range(len(rd_matrix)):
        for j in range(len(rd_matrix[i])):
            real = rd_matrix[i][j][0]
            real_mantissa, real_exponent = split_scientific_str(real)
            imag = rd_matrix[i][j][1]
            imag_mantissa, imag_exponent = split_scientific_str(imag)
            if real_exponent > imag_exponent:
                real_mantissa *= 10 ** (real_exponent - imag_exponent)
                value[i][j] = 10 * (log10(real_mantissa ** 2 + imag_mantissa ** 2 + eps) +  2 * imag_exponent)
            elif real_exponent < imag_exponent:
                imag_mantissa *= 10 ** (imag_exponent - real_exponent)
                value[i][j] = 20 * (log10(sqrt(real_mantissa ** 2 + imag_mantissa ** 2 + eps)) + real_exponent)
            else:
                value[i][j] = 20 * (log10(sqrt(real_mantissa ** 2 + imag_mantissa ** 2 + eps)) + real_exponent)

    range_axis = data['range_axis']
    velocity_axis = data['velocity_axis']

    if verbose:
        print(f"RD矩阵形状: {rd_matrix.shape}")
        print(f"距离轴长度: {len(range_axis)}")
        print(f"速度轴长度: {len(velocity_axis)}")

    plt.figure(figsize=(10, 8))

    X, Y = np.meshgrid(velocity_axis, range_axis)
    plt.pcolormesh(X, Y, value, shading='auto', cmap='jet')
    plt.xlim(-30, 30)
    plt.colorbar(label='幅度')
    plt.xlabel('速度 (m/s)')
    plt.ylabel('距离 (m)')
    plt.title('距离-多普勒图 (RD Map)')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    if save_path:
        check_path(save_path)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [ ]:
label = 6
batch = 3115
frame = 59

data = read_matlab_v73_file(label, batch, frame)
visualize_rd_matrix(data, verbose=True)

# 批量处理

In [6]:
import glob
from utils.dataset import read_mat


data_root = "E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results"
label_dirs = glob.glob(f"{data_root}/Label_*")
image_sizes = []
for label_dir in label_dirs:
    batch_dirs = glob.glob(f"{label_dir}/Batch_*")
    for batch_dir in batch_dirs:
        frame_dirs = glob.glob(f"{batch_dir}/Frame_*")
        for frame_dir in frame_dirs:
            data = read_mat(os.path.join(frame_dir, "MTD_result.mat"))
            image_size = data['rd_matrix'].shape
            if image_size not in image_sizes:
                image_sizes.append(image_size)
print(image_sizes)

[(31, 226), (31, 360), (31, 900), (31, 570), (31, 142)]


In [7]:
for label_dir in label_dirs:
    batch_dirs = glob.glob(f"{label_dir}/Batch_*")
    for batch_dir in batch_dirs:
        frame_dirs = glob.glob(f"{batch_dir}/Frame_*")
        for frame_dir in frame_dirs:
            data = read_mat(os.path.join(frame_dir, "MTD_result.mat"))
            image_size = data['rd_matrix'].shape
            if image_size == (31, 900):
                print(frame_dir)

E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_104\Frame_89
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_104\Frame_90
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_104\Frame_91
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_104\Frame_92
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_105\Frame_107
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_105\Frame_108
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_108\Frame_1
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_108\Frame_2
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_108\Frame_3
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_108\Frame_4
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_108\Frame_5
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_108\Frame_6
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_109\Frame_17
E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results\Label_1\Batch_109\Frame

In [3]:
label = 1
batch = 1
frame = 1
file_path = f"E:/DataSets/挑战杯_揭榜挂帅_CQ-08赛题_数据集/MTD_Results/Label_{label}/Batch_{batch}/Frame_{frame}/Target_Detection.mat"
with h5py.File(file_path, 'r') as f:
    var_names = list(f.keys())
    data = {}

    for var_name in var_names:
        # HDF5 中的数据是按列优先存储的，而 NumPy 是按行优先，因此需要转置
        if isinstance(f[var_name], h5py.Dataset):
            data[var_name] = np.array(f[var_name]).T
data

{'range': array([[285.]]), 'velocity': array([[15.35]])}

285.0